# Notebook

In [ ]:
//let startDate = DateTime(2021, 10, 31).ToUniversalTime()
//let endDate = DateTime(2021, 11, 01).ToUniversalTime()
let startDate = DateTime(2021, 11, 01).ToUniversalTime()
let endDate = DateTime(2021, 11, 02).ToUniversalTime()

let ts = endDate - startDate
let hoursCount = ts.Days * 24 + ts.Hours
let minutesCount = hoursCount * 60

let baseIntervals = [5; 15; 60]

let toIntervals n =
    baseIntervals
    |> List.filter (fun x -> n % x = 0)

let intervals =
    [0 .. 5 .. minutesCount]
    |> List.tail
    |> List.map toIntervals
    |> List.concat

intervals
|> List.length
|> printfn "%i"

intervals
|> List.map (sprintf "%i")
|> String.concat " "

425


5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60 5 5 5 15 5 5 5 15 5 5 5 15 5 5 5 15 60

In [ ]:
string Test(ReadOnlySpan<char> span) {
    var buffer = new Span<char>(new char[1]);
    span[.. 1].ToLower(buffer, System.Globalization.CultureInfo.CurrentCulture);
    return string.Concat(buffer, span[1 ..]);
}

var x = Test("SomeProp");
x

someProp

In [ ]:
let (a, b) = (1, 2)
a + b

3

In [ ]:
<p style="color: red;">hello</p>

In [ ]:
#r "nuget: Oracle.EntityFrameworkCore"

open System
open System.ComponentModel.DataAnnotations.Schema
open Microsoft.EntityFrameworkCore
open Microsoft.Extensions.Configuration

let connectionString = "Data Source=(DESCRIPTION = (ADDRESS = (PROTOCOL = TCP)(HOST = localhost)(PORT = 1521))(CONNECT_DATA = (SERVER = DEDICATED)(SERVICE_NAME = XE)));User ID=spr;Password=spr;"

[<CLIMutable>]
[<Table("OBJ_RANKING")>]
type Ranking = { 
      ID : int
      CODE : string
    }

type ObjDbContext() =
    inherit DbContext()

    [<DefaultValue>] val mutable rankings : DbSet<Ranking>    
    member this.Rankings with get() = this.rankings and set v = this.rankings <- v

    override this.OnConfiguring optionsBuilder =        
        optionsBuilder.UseOracle(connectionString) |> ignore

let ctx = new ObjDbContext()

let result = query {
  for r in ctx.Rankings do
  select r.CODE
  take 10
}

result


In [ ]:
open System
open System.Linq
open System.Text.RegularExpressions
open System.IO

let inputDir = @"C:\DEV\Git\SPR\spr-backend"
let outputDir = @"C:\DEV\Git\SPR"
let files = Directory.GetFiles(inputDir, "*Map.cs", SearchOption.AllDirectories)
let getModelAndTableName file =
    let fileContent = File.ReadAllText file
    let find regex = Regex.Match(fileContent, regex).Groups.[1].Value
    let modelName = find "ClassMap<(\w+)>"
    let tableName = find "Table\(\"(\w+)\"\)"
    let tableName =
        if String.IsNullOrEmpty tableName then 
            find " TableName = \"(\w+)\";"
        else
            tableName
    (modelName, tableName)

let data = 
    files
    |> Seq.map getModelAndTableName
    |> Seq.filter (fun (model, table) -> not (String.IsNullOrEmpty model || String.IsNullOrEmpty table))
    |> Seq.map (fun (model, table) -> $"select '{model}' as model_name, '{table}' as table_name from dual ")
    |> Array.ofSeq
    |> String.concat "union\n"

File.WriteAllText(Path.Combine(outputDir, "foo.txt"), data)


In [ ]:
#r "nuget: FSharp.Data, 4.1.1"

open System
open FSharp.Data

let query = "{ \"query\": \"{ continents { name } }\" }"
let result = Http.RequestString
              (
                url = "https://countries.trevorblades.com",
                httpMethod = "POST",
                headers = [ HttpRequestHeaders.ContentType "application/json" ],
                body = TextRequest query
              )
printfn $"Query: {query}\nResult: {result}"

In [ ]:
1 :: [2]

In [ ]:
open System

type RoundingWorkflow(sigDigs : int) =
    let round (x : float) = Math.Round(float x, sigDigs)

    member this.Bind (result : float, rest : float -> float) =
        let result' = round result
        rest result'

    member this.Return (x : float) : float =
        round x

let withPrecision sigDigs = RoundingWorkflow(sigDigs)

let test =
    withPrecision 3 {
      let! x = 1.0 / 3.0
      return x
    }

test


In [ ]:
let isPrime n =
    match n with
    | 1 -> false
    | 2 -> true
    | _ when n % 2 = 0 || [3 .. 2 .. int (sqrt (float n)) + 1] |> Seq.exists (fun x -> n % x = 0) -> false
    | _ -> true

let prime n =
    Seq.initInfinite ((+) 1) |> Seq.filter isPrime |> Seq.item (n - 1)

let p = prime 10001

p

In [ ]:
console.log(2 ** 10);

In [ ]:
var x = 1;
x + 1

In [ ]:
docker ps -a